# Cichlid genome variant calling

### Step 1 : align the reads

Download the *Oreochromis niloticus* (Nile tilapia) genome :

>[ftp://hgdownload.cse.ucsc.edu/goldenPath/oreNil2/bigZips/README.txt](ftp://hgdownload.cse.ucsc.edu/goldenPath/oreNil2/bigZips/README.txt)


>Jan. 2011 (Nile tilapia/oreNil2) assembly of the nile tilapia genome
>(oreNil2, Broad Institute of MIT and Harvard Orenil1.1 (GCA_000188235.1))

We use [`bowtie2`](http://bowtie-bio.sourceforge.net/bowtie2/index.shtml) 
to align the reads and [`samtools`](https://samtools.github.io/) to generate
sorted BAM files.

In [ ]:
# kill this before it writes too much data -- we just want the read group
!zcat ../biggerrun/Checkmate_NoIndex_L007_R1_001.fastq.gz > R1.fasta

!readgroupID=`head -n 1 R1.fastq | cut -d ":" -f 3,4 | sed s/:/-/g`

!rm R1.fastq

!bowtie2-build oreNil2.fa oreNil2

!bowtie2 -x oreNil2 -1 \
../biggerrun/Checkmate_NoIndex_L007_R1_001.fastq.gz -2 \
../biggerrun/Checkmate_NoIndex_L007_R3_001.fastq.gz -q --phred33 -S \
checkmate.bt2.sam --un-conc checkmate.unmapped.fastq --end-to-end \
--sensitive --fr -X 500 -I 0 --rg-id $readgroupID --rg SM:checkmate \
--rg PL:ILLUMINA --rg LB:checkmate -p 16
        
!samtools view -bS checkmate.bt2.sam > checkmate.bt2.bam

!samtools sort -@ 24 -m 4G checkmate.bt2.bam checkmate.bt2.sorted

The output for the alignment step was... 

    208652558 reads; of these:
      208652558 (100.00%) were paired; of these:
        114535729 (54.89%) aligned concordantly 0 times
        73929838 (35.43%) aligned concordantly exactly 1 time
        20186991 (9.67%) aligned concordantly >1 times
        ----
        114535729 pairs aligned concordantly 0 times; of these:
          5023993 (4.39%) aligned discordantly 1 time
        ----
        109511736 pairs aligned 0 times concordantly or discordantly; of
    these:
          219023472 mates make up the pairs; of these:
            154234827 (70.42%) aligned 0 times
            37425398 (17.09%) aligned exactly 1 time
            27363247 (12.49%) aligned >1 times
    63.04% overall alignment rate

### Step 2 : clean up the alignments

We used [`picard`](http://broadinstitute.github.io/picard/) to call the
variants. Here are the commands run for "checkmate," the fish from 
lake Victoria.

For the "checkmate" genome, I used a current version of picard 
(1.130) instead of the ancient version used in HPC submission scripts 
used for the other genomes (1.80). The calling semantics are slightly
different, but the options are the same.

In [ ]:
!java -Xmx4g -Djava.io.tmpdir=./temp -jar \
../../pkg/picard-tools-1.130/picard.jar MarkDuplicates \
INPUT=checkmate.bt2.sorted.bam OUTPUT=checkmate.bt2.dedup.bam \
METRICS_FILE=checkmate.bt2.metrics.txt TMP_DIR=/tmp/ \
ASSUME_SORTED=true MAX_FILE_HANDLES_FOR_READ_ENDS_MAP=1000 \
READ_NAME_REGEX="[a-zA-Z0-9]+-[a-zA-Z0-9]+:[0-9]+:[a-zA-Z0-9]+:[0-9]+:([0-9]+):([0-9]+):([0-9]+)"

!samtools faidx oreNil2.fa

!java -jar ../../pkg/picard-tools-1.130/picard.jar \
CreateSequenceDictionary R=oreNil2.fa O=oreNil2.dict

!samtools index checkmate.bt2.dedup.bam

!java -Xmx4g -Djava.io.tmpdir=/tmp -jar ../../pkg/GenomeAnalysisTK.jar \
-T DepthOfCoverage -R oreNil2.fa -I checkmate.bt2.dedup.bam -o \
depth.checkmate --minBaseQuality 17 --minMappingQuality 10 \
--summaryCoverageThreshold 15 --countType \
COUNT_FRAGMENTS_REQUIRE_SAME_BASE

!java -Xmx4g -Djava.io.tmpdir=/tmp -jar ../../pkg/GenomeAnalysisTK.jar \
-T RealignerTargetCreator -R oreNil2.fa -I checkmate.bt2.dedup.bam -nt \
16 -o target_intervals.list

!java -Xmx4g -Djava.io.tmpdir=/tmp -jar ../../pkg/GenomeAnalysisTK.jar \
-T IndelRealigner -R oreNil2.fa -I checkmate.bt2.dedup.bam \
-targetIntervals target_intervals.list -o checkmate.dedup.realign.bam

### Step 3 : call the variants

Sorted, deduped and realigned BAM files were created for the other
cichlid genomes using the same method described above for the "checkmate"
genome. The `UnifiedGenotyper`in `picard` was then used to call the
variants, and [`bcftools`](https://samtools.github.io/bcftools/) was used
to pull out the SNPs for the speices pairs in lakes Kivu, Victoria and Mweru.

In [ ]:
!for i in $(ls *.bam); do samtools index $i; done \
java -Xmx4g -Djava.io.tmpdir=/tmp -jar ../../pkg/GenomeAnalysisTK.jar \
-T UnifiedGenotyper -R oreNil2.fa -I 64253.dedup.realign.bam -I \
64571.dedup.realign.bam -I 64767.dedup.realign.bam -I \
78357.dedup.realign.bam -I 78584.dedup.realign.bam -I \
80344.dedup.realign.bam -I 81032.dedup.realign.bam -I \
81342.dedup.realign.bam -I checkmate.dedup.realign.bam -o \
cichlid.wgs.vcf -nt 16 -nct 3 --output_mode EMIT_ALL_CONFIDENT_SITES \
-stand_call_conf 30 -stand_emit_conf 30 --min_base_quality_score 17 \
--genotype_likelihoods_model BOTH

!bcftools/bcftools view cichlid.wgs.vcf -o SNPs_only.kivu.vcf -O v -x -s 64571,64253

!bcftools/bcftools view cichlid.wgs.vcf -o SNPs_only.victora.vcf -O v -x -s checkmate,80344

!home/russell/pkg/bcftools/bcftools view cichlid.wgs.vcf -o SNPs_only.mweru.vcf -O v -x -s 78357,78584

### Continue on to the ulakes notebook...